In [ ]:
import pandas as pd
import numpy as np
import csv

# KYC Feature Engineering

The purpose of this notebook is to add wildlife trafficking indicators to the KYC data. Each of the indicators takes on a value $[0,1]$. These flags only depend on the characteristics of neighbours within 1-hop of the node. 

## Profile-Based Indicators
Per the [Fintrac Operational Alert](https://fintrac-canafe.canada.ca/intel/operation/oai-wildlife-eng), one indicator is "An individual is the owner, operator, employee or associated with an industry that could be used to facilitate illegal wildlife trade (e.g., import/export of goods, fisheries wholesaler, pet store, freight company, animal control)." (**G**)

The [Financial Crime Academy](https://financialcrimeacademy.org/wildlife-trade-risk-indicators-financial/?fbclid=IwAR1XSw09Vtl4mjOOQj_eTFuqZ_GKqM-SPsCJwQKcyFb-XWU4O6nO8zBo3JU) adds that another "indicator relates to activity involving politically exposed persons and wealthy businessmen/women, particularly those with environmental, game, or forestry oversight or environmental or wildlife-related businesses." (**G**)

We have therefore come up with the following indicators based on KYC occupation data:
- `wealth_exp` 
    - binary 
    - 1 if the client's occupation involves frequent exposure to wealthy people, 0 otherwise.
- `animal` 
    - binary 
    - 1 if the client's occupation involves working with animals, 0 otherwise.
- `int_trade` 
    - binary 
    - 1 if the client works international trade, 0 otherwise.

In [35]:
# Adding the occupation-based indicators to raw data
raw = pd.read_csv('./raw/kyc.csv')
occ = pd.read_csv('./occupation_list.csv')

merged = raw.merge(occ, 'left', on='Occupation')

# Re-order columns
cols = list(merged.columns)
cols.append(cols.pop(cols.index('label')))
merged = merged[cols]

# Export
display(merged.head(3))
merged.to_csv('kyc_adj.csv', index=False)

,Name,Gender,Occupation,Age,Tenure,cust_id,wealth_exp,animal,int_trade,label
0,JENNIFER WELLS,female,Architect,45.0,13.0,CUST82758793,0,0,0,0
1,ANTHONY ADAMS,male,Musician,52.0,8.0,CUST69248708,2,0,0,0
2,DENISE LEWIS,female,Jewelry Dealer,43.0,11.0,CUST67222818,1,0,2,0


## Transaction-based Indicators

[Fintrac Operational Alert](https://fintrac-canafe.canada.ca/intel/operation/oai-wildlife-eng) gives some transaction based indicators:
- Transaction message indicators
    - Message makes reference to species or animal parts of concern (e.g. ivory, tusk, scales, shark fin, tortoise, gecko). (**G**)
    - Message makes reference to traditional medicine (**I**)
    - Message (or email addresses) species the role of the beneficiary of the funds as being related to wildlife trafficking (e.g. poacher, coordinator, supplier, breeder, trader, courier)

- Outgoing transaction indicators
    - Wire-transfers sent to individuals and/or entities located in jurisdictions of concern for wildlife crime (e.g., China, Hong Kong, Australia, and Sub-Saharan Africa).(**I**)
    - Transactions sent to entities operating in the traditional medicine market. (**I**)
    - Transactions sent to shipping/postal services/cargo services (**E**)
    - Transactions sent to animal related business that are inconsistent with the clients profile (**I**)
    
- Incoming transaction indicators
    - Transactions recieved from animal related business that are inconsistent with the clients profile (**E**)
    - Transactions recieved that reference animal parts of concern from Canada (**E**)

The [Financial Crime Academy](https://financialcrimeacademy.org/wildlife-trade-risk-indicators-financial/?fbclid=IwAR1XSw09Vtl4mjOOQj_eTFuqZ_GKqM-SPsCJwQKcyFb-XWU4O6nO8zBo3JU) adds the following general indicators:
- Transaction indicators
    - Transactions to people involved in wildlife (larger is more suspicious)
    - Large transactions 
- Cash transaction indicators
    - Large deposits by gov't, customs, border control
    - Large deposits, or multiple small deposits, from people working with wildlife



## Other Individual Indicators
[Fintrac Operational Alert](https://fintrac-canafe.canada.ca/intel/operation/oai-wildlife-eng)
- Media and/or law enforcement information links client or transacting parties to the illegal wildlife trade.(**G**)
- The Individual or an entity has open source information linking them to the possession and/or advertising the sale of species of concern for illegal wildlife trade, particularly endangered species (e.g., geckos, turtles, and tortoises).(**G**)
- 

# KYC Feature Engineering - Graph Structure
- Recieving numerous with wildlife-trade references from several parties, followed by outgoing transfers to individuals who also recieved similar e-transfers from other parties, with very few or no other transactions in their account.(**G**)
- 